<a href="https://colab.research.google.com/github/Hamza-Chekireb/falcon-llm-t4-gpu-chat-evaluation/blob/main/Falcon_7b_Model_MVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.Import Requirements

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
! pip install langchain
# !pip install faiss-cpu
! pip install transformers
# !pip install auto_gptq
# !pip install -qqq InstructorEmbedding==1.0.1
! pip install pypdf
! pip install sentence_transformers
! pip install transformers
# This library helps you run PyTorch code on any device.
# It simplifies the code needed for using multiple GPUs, TPUs, or mixed precision
! pip install accelerate
# This is a library for manipulating tensors.
# It provides a more readable and expressive way to handle tensors in deep learning and scientific computing
! pip install einops
! pip install xformers
! pip install langchain
! pip install langchain_community
# !pip install -U langchain-community
! pip install chromadb

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from transformers import AutoTokenizer, TextStreamer, pipeline
from langchain import HuggingFacePipeline
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline # AutoModelForCausalLM is used to add layer for application like QA
import langchain
from langchain import HuggingFacePipeline

### 2.Creat the Vectore Data Base

In [ ]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

DATA_PATH = '/content/'
DB_FAISS_PATH = 'vectorstore/db_chroma'

# Create vector database
def create_vector_db():
    loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                   chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    # embeddings = HuggingFaceInstructEmbeddings(
    # model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
# )

    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': DEVICE})

    # 4. Creat Vector Data Base
    db = Chroma.from_documents(texts,embeddings,persist_directory=DB_FAISS_PATH)
    # store the db in repertory
    db.persist()

if __name__ == "__main__":
    create_vector_db()

###3.Initiate a model

#### 3.3. Falcon 7B

In [ ]:
model = "tiiuae/falcon-7b-instruct" #tiiuae/falcon-40b-instruct

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    truncation=True,
    max_new_tokens=256,
    max_length=1024,
    do_sample=True,
    # top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    return_full_text=False
)

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs={"truncation": True, "temperature":0.1})

In [ ]:
llm.invoke("Can you tell me a fact about the moon ?\n?")

In [ ]:
# llm("Can you tell me a fact about the moon ?")

### 4.Output Function

 #### 4.1. create_retrieval_chain

######4.1.1.Importation

In [ ]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


######4.1.2. Retriever

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': DEVICE})
VDBPath = 'vectorstore/db_chroma'
vdb_connection = Chroma(persist_directory=VDBPath,  embedding_function=embeddings)
# Create new connection to VDB
retriever = vdb_connection.as_retriever(search_kwargs={'k': 2})

### Contextualize question to be understood without need to chat history

In [ ]:
# # Contextualize question from history
# contextualize_q_system_prompt = (
# """Given this chat history and the latest user question which might reference context in the chat history,\
#  formulate a standalone question which can be understood\
#   without the chat history. Do NOT answer the question, just\
#   reformulate it if needed and otherwise return it as is.""")
# contextualize_q_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", contextualize_q_system_prompt),
#         MessagesPlaceholder("chat_history"),
#         ("human", "{input}"),
#     ]
# )

In [ ]:
# # Function creates a retriever that uses chat history and a language model to
# # retrieve relevant documents for answering user questions.
# history_aware_retriever = create_history_aware_retriever(
#     llm, retriever, contextualize_q_prompt
# )

######4.1.3. Answering question prompt

In [ ]:
# # Answer question
# qa_system_prompt = (
#     "Use "
#     "the  context and the following chat history to answer and assist the Human. "
#     "If you don’t know the answer to a question, just say that you do not know. "
#     "The response should be short, concise and exhaustive. "
#     "\n Context : {context}"
#     # "\n Chat History : {chat_history}"
# )

# qa_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", qa_system_prompt),
#         MessagesPlaceholder("chat_history"),
#         ("human", "{input}"),
#     ]
# )

In [ ]:
# # Below we use create_stuff_documents_chain to feed all retrieved context
# # into the LLM. Note that we can also use StuffDocumentsChain and other
# # instances of BaseCombineDocumentsChain.
# question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
# rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
# #output function
# def final_result(query, history):
#   result = rag_chain.invoke({"input": query, "chat_history": history})
#   history.append(('human', query))
#   history.append(('assistant', result['answer']))

#   def keep_last_n_tuples(lst, n=4):
#     return lst[-n:] if len(lst) > n else lst

#   history = keep_last_n_tuples(history)
#   return result['answer']

In [ ]:
# history=[]
# while True:
#     query = input("What's on your mind? (Type 'exit' to end): ")
#     if query.lower() == "exit":
#         # history.clear()
#         break

#     res = final_result(query,history)
#     print(res)

In [ ]:
# query = "I would like the contacts of a FRONTEND DEVELOPER"
# # history = []
# result = rag_chain.invoke({"input": query, "chat_history": history})
# history.append(('human', query))
# history.append(('system', result['answer']))

# def keep_last_n_tuples(lst, n=4):
#   return lst[-n:] if len(lst) > n else lst

# history = keep_last_n_tuples(history)
# result['answer']

In [ ]:
# print(result['answer'])

### RAG From LangChain

In [ ]:
# import bs4
# from langchain.chains import create_history_aware_retriever, create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_chroma import Chroma
# from langchain_community.chat_message_histories import ChatMessageHistory
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_core.chat_history import BaseChatMessageHistory
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# ### Contextualize question ###
# contextualize_q_system_prompt = """Given a chat history and the latest user question \
# which might reference context in the chat history, formulate a standalone question \
# which can be understood without the chat history. Do NOT answer the question, \
# just reformulate it if needed and otherwise return it as is."""

# contextualize_q_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", contextualize_q_system_prompt),
#         MessagesPlaceholder("chat_history"),
#         ("human", "{input}"),
#     ]
# )

# history_aware_retriever = create_history_aware_retriever(
#     llm, retriever, contextualize_q_prompt
# )


# ### Answer question ###
# qa_system_prompt = """You are an assistant for question-answering tasks. \
# Use the following pieces of retrieved context to answer the question. \
# If you don't know the answer, just say that you don't know. \
# Use three sentences maximum and keep the answer concise.\

# {context}"""
# qa_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", qa_system_prompt),
#         MessagesPlaceholder("chat_history"),
#         ("human", "{input}"),
#     ]
# )

# question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


# rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# ### Statefully manage chat history ###
# store = {}


# def get_session_history(session_id: str) -> BaseChatMessageHistory:
#     if session_id not in store:
#         store[session_id] = ChatMessageHistory()
#     return store[session_id]


# conversational_rag_chain = RunnableWithMessageHistory(
#     rag_chain,
#     get_session_history,
#     input_messages_key="input",
#     history_messages_key="chat_history",
#     output_messages_key="answer",
# )

##### 4.2.Simple Chain

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory


In [ ]:
# Create new connection to VDB

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': DEVICE})
VDBPath = 'vectorstore/db_chroma'
vdb_connection = Chroma(persist_directory=VDBPath,  embedding_function=embeddings)


In [ ]:
#output function
def final_result(question):
  docs = vdb_connection.similarity_search(question,k=2)
  context = "\n\n ".join([doc.page_content for doc in docs])

  promp_template = "Answer this Question ({question}) based only on the given context.\n\n" \
+ f"Context: {context}\n\n" + "Question : {question}\n"

  promp_template_for_processing = f"Answer this Question ({question}) based only on the given context.\n\n" \
+ f"Context: {context}\n\n" + f"Question : {question}\n"

  prompt =PromptTemplate(template=promp_template, input_variables=["question"])
  chain = LLMChain(llm=llm, prompt=prompt, verbose=False)

  result = chain.run(question=question).replace(promp_template_for_processing,"")

  return result

In [ ]:
while True:
    question = input("Ask me a question? (Type 'exit' to end): ")
    if question.lower() == "exit":
        break
    result = final_result(question)
    print(result)